Ejemplo de uso sin usar programacion orientada a objetos:


Nota: en caso de requerir ayuda sobre las funciones puede abir la libreta llamada "Funciones".

In [ ]:
import os
import urllib.request
import pandas as pd
import tarfile
from collections import defaultdict
from pathlib import Path
import requests
from bs4 import BeautifulSoup

def descargar_archivo(url, directorio):
  '''Descarga un archivo desde la URL especificada y lo guarda en el directorio indicado. Si el archivo ya existe, no se descarga de nuevo.'''
  archivo = os.path.join(directorio, os.path.basename(url))
  os.makedirs(os.path.dirname(archivo), exist_ok=True)
  if not os.path.exists(archivo):
    urllib.request.urlretrieve(url, archivo)
    print(f"Descargado en {archivo}")
  else:
    print(f"El archivo ya existe.")
  return archivo

def descomprimir_tar_gz(archivo, directorio):
  '''Descomprime un archivo .tar.gz en el directorio especificado.'''
  with tarfile.open(archivo, "r:gz") as tar:
    tar.extractall(path=directorio)
    print(f"Descomprimido en {directorio}.")

def leer_puntuaciones(directorio):
  '''Lee las puntuaciones de los archivos en el directorio dado y las organiza por identificador.'''
  puntuaciones = defaultdict(list)
  for subdir, _, files in os.walk(directorio):
    for file_name in files:
      try:
        identificador, puntuacion_str = file_name.split('_')
        puntuacion = int(puntuacion_str.split('.')[0])
        puntuaciones[identificador].append(puntuacion)
      except (ValueError, IndexError):
        continue
  return puntuaciones

def calcular_promedios(puntuaciones):
  '''Calcula los promedios de las puntuaciones para cada identificador y devuelve un conjunto de (identificador, promedio).'''
  return {identificador: sum(scores) / len(scores) for identificador, scores in puntuaciones.items()}  # retornar un diccionario con promedios

def obtener_top_n(promedios, n=10, mejor=True):
  '''Obtiene las top n películas con mejor o peor promedio. Devuelve una lista de (película, promedio) sin repeticiones.'''
  sorted_movies = sorted(promedios.items(), key=lambda x: x[1], reverse=mejor)
  peliculas_unicas = []
  unicas = set()
  for pelicula, promedio in sorted_movies:
    if pelicula not in unicas:
      peliculas_unicas.append((pelicula, promedio))
      unicas.add(pelicula)
    if len(peliculas_unicas) == n:
      break
  return peliculas_unicas

def procesar_url(url):
  '''Procesa una URL para eliminar la parte que incluye '/usercomments' y retorna la parte relevante.'''
  return url.split('/usercomments')[0]

def obtener_nombre_pelicula(url):
  '''Obtiene el nombre de la película a partir de la url utilizando scraping. Retorna el título o un mensaje de error si no se encuentra.'''
  headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"
  }
  response = requests.get(url, headers=headers)
  if response.status_code == 403:
    return "Acceso denegado (403 Forbidden)"

  soup = BeautifulSoup(response.content, 'html.parser')
  title_tag = soup.find('title')
  if title_tag:
    return title_tag.text.replace(' - IMDb', '').strip()
  else:
    return "Título no encontrado"

def crear_dataframe(peliculas, urls_path):
  '''Crea un DataFrame de pandas a partir de las películas y las urls. Retorna un DataFrame con la url, el nombre de la película y su promedio.'''
  urls = Path(urls_path).read_text().splitlines()
  data = []
  nombres_vistos = set()
  for identificador, promedio in peliculas:
    url = procesar_url(urls[int(identificador) - 1])
    nombre_pelicula = obtener_nombre_pelicula(url) l
    if nombre_pelicula not in nombres_vistos:
      nombres_vistos.add(nombre_pelicula)
      data.append({'url': url, 'Película': nombre_pelicula, 'Promedio': promedio})
  return pd.DataFrame(data)

def imprimir_peliculas(titulo, peliculas):
  '''Imprime en consola el título y la lista de películas con su promedio y URL.'''
  print(titulo)
  for pelicula in peliculas:
    print(f"Película: {pelicula['Película']} || Promedio: {pelicula['Promedio']:.2f} || url: {pelicula['url']}")  # Imprimir detalles de la película

def analizar_peliculas(directorio_pos, directorio_neg, urls_pos, urls_neg):
  '''Coordina el proceso de análisis de películas, incluyendo la lectura de puntuaciones, cálculo de promedios y la impresión de resultados.'''
  calificaciones_pos = leer_puntuaciones(directorio_pos)
  calificaciones_neg = leer_puntuaciones(directorio_neg)
  promedios_pos = calcular_promedios(calificaciones_pos)
  promedios_neg = calcular_promedios(calificaciones_neg)
  mejores_peliculas = obtener_top_n(promedios_pos, n=10)
  peores_peliculas = obtener_top_n(promedios_neg, n=10, mejor=False)
  lista_mejores = crear_dataframe(mejores_peliculas, urls_pos)
  lista_peores = crear_dataframe(peores_peliculas, urls_neg)
  while len(lista_mejores) < 10:
    n_a_row = pd.DataFrame({'url': [''], 'Película': ['N/A'], 'Promedio': [0]})
    lista_mejores = pd.concat([lista_mejores, n_a_row], ignore_index=True)
  while len(lista_peores) < 10:
    n_a_row = pd.DataFrame({'url': [''], 'Película': ['N/A'], 'Promedio': [0]})
    lista_peores = pd.concat([lista_peores, n_a_row], ignore_index=True)
  imprimir_peliculas("Las 10 películas mejor calificadas:\n", lista_mejores.to_dict(orient='records'))
  imprimir_peliculas("Las 10 películas peor calificadas:\n", lista_peores.to_dict(orient='records'))


url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
directorio = "./downloads/basedatos/aclImdb"
directorio_pos = './downloads/basedatos/aclImdb/aclImdb/train/pos'
directorio_neg = './downloads/basedatos/aclImdb/aclImdb/train/neg'
urls_pos = './downloads/basedatos/aclImdb/aclImdb/train/urls_pos.txt'
urls_neg = './downloads/basedatos/aclImdb/aclImdb/train/urls_neg.txt'

archivo = descargar_archivo(url, directorio)
descomprimir_tar_gz(archivo, directorio)
analizar_peliculas(directorio_pos, directorio_neg, urls_pos, urls_neg)



El archivo ya existe.
Descomprimido en ./downloads/basedatos/aclImdb.
Las 10 películas mejor calificadas:

Película: The Odd Couple (1968) || Promedio: 10.00 || url: http://www.imdb.com/title/tt0063374
Película: Seven Pounds (2008) || Promedio: 10.00 || url: http://www.imdb.com/title/tt0814314
Película: A River Runs Through It (1992) || Promedio: 10.00 || url: http://www.imdb.com/title/tt0105265
Película: Beat Street (1984) || Promedio: 10.00 || url: http://www.imdb.com/title/tt0086946
Película: Vivah (2006) || Promedio: 10.00 || url: http://www.imdb.com/title/tt0494290
Película: Emma (1996) || Promedio: 10.00 || url: http://www.imdb.com/title/tt0116191
Película: The Grilling (1981) || Promedio: 10.00 || url: http://www.imdb.com/title/tt0082436
Película: Gunga Din (1939) || Promedio: 10.00 || url: http://www.imdb.com/title/tt0031398
Película: Bonanza (TV Series 1959–1973) || Promedio: 10.00 || url: http://www.imdb.com/title/tt0052451
Película: The Little American (1917) || Promedio: 10